# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the reciprocal of the l2-norm between the item embeddings

In [1]:
name = "ItemCFEmbed";

In [2]:
using NBInclude
@nbinclude("ItemCollaborativeFilteringBase.ipynb");

## Compute cosine correlations

In [3]:
function get_embed_matrix_outdir(power)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(power))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $power-norm"
    item_embeddings = vcat(
        [collect(read_params("MatrixFactorization.$K")["A"]') for K in [10, 20, 40]]...,
    )
    training = get_residuals("training", residual_alphas)
    S = zeros(maximum(training.item), maximum(training.item))

    kernel = item_embeddings
    p = power
    @showprogress for j = 1:size(S)[1]
        dists = map(x -> norm(x, p), eachslice(kernel .- kernel[:, j], dims = 2))
        S[:, j] = 1 ./ dists
    end

    # only the diagonal can be Inf
    for i = 1:size(S)[1]
        for j = 1:size(S)[2]
            if S[i, j] == Inf && i != j
                S[i, j] = 1e8
            end
        end
    end

    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [4]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
item_alphas = ["ItemCF.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
item_cf_params = [
    cf_params(
        name = "ItemCFEmbed.$K",
        training_residuals = ["UserItemBiases"],
        validation_residuals = ["UserItemBiases"],
        neighborhood_type = "abs",
        S = get_embed_matrix_outdir(2),
        K = K,
        λ = [2.4182964143573114, 1.0364205793963885, 0.000497907758177069],
    ) for K in downcast_to_int.([2^10])
];

## Train models

In [ ]:
for param in item_cf_params
    optimize_model(param)
end

Progress:  82%|██████████████████████▏    |  ETA: 0:01:08 ( 0.38  s/it)